In [ ]:
from pathlib import Path
from tqdm.autonotebook import tqdm
import json

import matplotlib.pyplot as plt
import cv2

from detectron2.structures.boxes import BoxMode

import sys
sys.path.append('../utils')

from train_eval import Evaluation

root_directory = Path("../data")
images_directory = root_directory / "images"
annots_directory =  root_directory / "annotations"
bbox_directory =  root_directory / "annotations-bbox"
masks_directory =  root_directory / "annotations-seg"

## Register Dataset

In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.data import MetadataCatalog, DatasetCatalog

path_to_train = annots_directory / 'modanet_instance_segmentation_train.json'
path_to_test = annots_directory / 'modanet_instance_segmentation_test.json'


register_coco_instances('modanet_instance_segmentation_train', {}, path_to_train, images_directory)
register_coco_instances('modanet_instance_segmentation_test', {}, path_to_test, images_directory)

MetadataCatalog.get('modanet_instance_segmentation_train').set(
    thing_classes=['bag', 'dress', 'footwear', 'skirt', 'top', 'sunglasses', \
                   'headwear', 'shorts', 'pants', 'belt', 'outer', 'scarf', 'boots']
)
train_metadata = MetadataCatalog.get('modanet_instance_segmentation_train')

In [ ]:
from detectron2.data.datasets import load_coco_json
dataset_dicts = load_coco_json("../data/annotations/modanet_instance_segmentation_train.json", "../data/images/")

Evaluation._show_n(dataset_dicts, train_metadata, predictions=False)

## Mask-RCNN

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
from detectron2 import model_zoo

import random
random.seed(600)

### Training

In [ ]:
import logging

from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import LazyConfig, instantiate
from detectron2.engine import (
    AMPTrainer,
    SimpleTrainer,
    default_setup,
    default_argument_parser,
    default_writers,
    hooks,
    launch
)
from detectron2.engine.defaults import create_ddp_model
from detectron2.evaluation import inference_on_dataset, print_csv_format
from detectron2.utils import comm

In [ ]:
def create_lazy_cfg(lazy_path, max_iter = 100):
    print(f"Creating {lazy_path}...\n")
    cfg = LazyConfig.load(model_zoo.get_config_file(lazy_path))
    default_setup(cfg, [])
    
    model = cfg.model
    model.backbone.bottom_up.stem.norm = \
    model.backbone.bottom_up.stages.norm = \
    model.backbone.norm = "FrozenBN"
    cfg.model = model
    
    cfg.train.amp.enabled = True
    cfg.train.output_dir = '../logs/exploration_test'
    cfg.train.max_iter = max_iter
    cfg.model.roi_heads.num_classes = 13
    cfg.dataloader.train.dataset.names = 'modanet_instance_segmentation_train'
    cfg.dataloader.test.dataset.names = 'modanet_instance_segmentation_test'
    
    return cfg

def do_train(cfg):
    model = instantiate(cfg.model)
    logger = logging.getLogger("detectron2")
    logger.info("Model:\n{}".format(model))
    model.to(cfg.train.device)

    cfg.optimizer.params.model = model
    optim = instantiate(cfg.optimizer)

    train_loader = instantiate(cfg.dataloader.train)

#     model = create_ddp_model(model, **cfg.train.ddp)
    trainer = (AMPTrainer if cfg.train.amp.enabled else SimpleTrainer)(model, train_loader, optim)
    checkpointer = DetectionCheckpointer(
        model,
        cfg.train.output_dir,
        optimizer=optim,
        trainer=trainer,
    )
    trainer.register_hooks(
        [
            hooks.IterationTimer(),
            hooks.LRScheduler(scheduler=instantiate(cfg.lr_multiplier)),
            hooks.PeriodicCheckpointer(checkpointer, **cfg.train.checkpointer)
            if comm.is_main_process()
            else None,
            hooks.EvalHook(cfg.train.eval_period, lambda: do_test(cfg, model)),
            hooks.PeriodicWriter(
                default_writers(cfg.train.output_dir, cfg.train.max_iter),
                period=cfg.train.log_period,
            )
            if comm.is_main_process()
            else None,
        ]
    )

    checkpointer.resume_or_load(cfg.train.init_checkpoint, resume=False)
    start_iter = 0
    trainer.train(start_iter, cfg.train.max_iter)

In [ ]:
def run_lazy(lazy_path):
    cfg = create_lazy_cfg(lazy_path)
    do_train(cfg)
    
new_model_lazy_path = 'new_baselines/mask_rcnn_R_50_FPN_400ep_LSJ.py'
run_lazy(new_model_lazy_path)

In [ ]:
from detectron2.config import get_cfg

model_yaml_path = 'COCO-InstanceSegmentation/mask_rcnn_R_101_FPN_3x.yaml'
# model_weights_path = 'detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl'

# cfg = LazyConfig.load()
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(model_yaml_path))
cfg.OUTPUT_DIR = '../logs/exploration_test'
cfg.DATASETS.TRAIN = ('modanet_instance_segmentation_train',)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.01

cfg.SOLVER.NUM_GPUS = 1
single_iteration = cfg.SOLVER.NUM_GPUS * cfg.SOLVER.IMS_PER_BATCH
iterations_for_one_epoch = len(dataset_dicts) / single_iteration
cfg.SOLVER.MAX_ITER = iterations_for_one_epoch * 2 # -> 2 epochs?   

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(model_yaml_path)
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

In [ ]:
# from detectron2.engine import DefaultTrainer

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
# trainer = DefaultTrainer(cfg)
# trainer.resume_or_load(resume=False)
# trainer.train()

### Evaluate

In [ ]:
eval_tools = Evaluation(cfg, "modanet_instance_segmentation_test")

In [ ]:
# eval_tools.run_coco_eval()

### Visualizations

In [ ]:
# from detectron2.engine import DefaultPredictor

# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')
# cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.4 # original 0.5
# cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4  # set the testing threshold for this model
# cfg.DATASETS.TEST = ('modanet_instance_segmentation_test', )
# predictor = DefaultPredictor(cfg)

train_metadata = MetadataCatalog.get('modanet_instance_segmentation_train')
test_dataset_dicts = DatasetCatalog.get('modanet_instance_segmentation_test')

In [ ]:
eval_tools.show_n(dataset_dicts=test_dataset_dicts, metadata=train_metadata, predictions = True)

In [ ]:
mapping = {k: v for k, v in enumerate(train_metadata.thing_classes)}
mapping

In [ ]:
test_annots = eval_tools.createAnnots(test_dataset_dicts, mapping = mapping, sample_ratio=2)

In [ ]:
# model_predictions = modanet_segmentation_dataset.create_annotation_set(
#     annotation_task = 'Instance Segmentation', name = 'exploration_preds_sampleratio2')

# modanet_segmentation_dataset.add_annotations(test_annots, annotation_set_id=model_predictions.id)

In [ ]:
modanet_segmentation_dataset.view()